In [2]:
out = open("topRNAs.fna", "w")
with open("GCF_000001405.40_GRCh38.p14_rna.fna") as f:
    chosen_rnas = 0
    names = []
    writing = True
    while chosen_rnas < 20:
        line = f.readline()
        if line[0] == ">":
            name = line.split()[0]
            if name not in names:
                chosen_rnas += 1
                writing = True
            else:
                writing = False
        if writing:
            out.write(line)

out.flush()
out.close()